In [51]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import string
import warnings
warnings.filterwarnings('ignore')
%config InlineBackend.figure_format='retina'

### Loading & Exploring Data

In [52]:
ipums = pd.read_csv('data/IPUMS_2019.csv')

In [53]:
ipums.columns

Index(['YEAR', 'SAMPLE', 'SERIAL', 'CBSERIAL', 'HHWT', 'CLUSTER', 'CPI99',
       'STATEFIP', 'COUNTYFIP', 'CITY', 'STRATA', 'GQ', 'OWNERSHP',
       'OWNERSHPD', 'OWNCOST', 'RENTGRS', 'HHINCOME', 'CILAPTOP', 'CIHISPEED',
       'PERNUM', 'PERWT', 'FAMSIZE', 'AGE', 'RACE', 'RACED', 'HISPAN',
       'HISPAND', 'EDUC', 'EDUCD', 'OCCSOC', 'INDNAICS', 'INCTOT', 'FTOTINC',
       'INCWAGE', 'MIGCOUNTY1', 'MOVEDIN', 'TRANTIME'],
      dtype='object')

In [54]:
ipums_titles = pd.read_csv('data/ind_indnaics_crosswalk_2000_onward_without_code_descriptions.csv')

In [55]:
ipums_titles = ipums_titles.iloc[2:]

In [56]:
ipums_titles = ipums_titles.iloc[:,10:]

In [57]:
county_info = pd.read_csv('data/county_to_regions_key - Sheet1.csv')

In [58]:
cost_of_living = pd.read_csv('data/united-way-reg-col.csv') # using united way real cost measure

In [59]:
cost_of_living = cost_of_living.iloc[0:11, :2]

In [60]:
cost_of_living

,Regions,Cost of Living
0,Bay Area,40083
1,Central Coast,34009
2,Central Valley,25668
3,Inland Empire,27549
4,Los Angeles,35245
5,Orange,38811
6,Redwood Coast,25001
7,Sacramento,28203
8,San Diego-Imperial,21109
9,Shasta / Cascades,24587


### Cleaning

In [61]:
ipums = ipums[['STATEFIP', 
               'COUNTYFIP',
              'INDNAICS',
              'PERWT',
              'INCWAGE']]

In [62]:
ca_ipums = ipums.loc[ipums['STATEFIP'] == 6].copy()

In [63]:
ca_ipums = ca_ipums.reset_index()

In [64]:
len(ca_ipums)

380091

In [65]:
def normalize_titles(col):
    col = col.astype(str)
    col = col.str.strip()
    col = col.str.lower()
    col = col.str.replace('&', 'and')
    col = col.apply(lambda x:''.join([i for i in x if i not in string.punctuation]))
    return col

In [66]:
ca_ipums['INDNAICS'] = normalize_titles(ca_ipums['INDNAICS'])

In [67]:
ipums_titles['2018 Onward ACS/PRCS INDNAICS CODE'] = normalize_titles(ipums_titles['2018 Onward ACS/PRCS INDNAICS CODE'])

### Merging on NAICS Codes

In [68]:
ca_ipums.columns.values

array(['index', 'STATEFIP', 'COUNTYFIP', 'INDNAICS', 'PERWT', 'INCWAGE'],
      dtype=object)

In [69]:
ipums_titles.columns.values

array(['2018 Onward ACS/PRCS INDNAICS CODE', 'Industry Title'],
      dtype=object)

In [70]:
ipums_merged = pd.merge(ca_ipums, ipums_titles,
                              left_on = 'INDNAICS', right_on = '2018 Onward ACS/PRCS INDNAICS CODE')

In [71]:
print(len(ca_ipums), len(ipums_titles), ipums_merged.shape)

380091 402 (224526, 8)


In [72]:
print(f'''TOTAL ROWS: {len(ipums_merged)}''')
print('\n')
for column in ipums_merged.columns.values:
    total = len(ipums_merged)
    nulls = ipums_merged[column].isna().sum()
    percentage = (nulls / total) * 100
    print(f'''{column}: {nulls} null values, {percentage:.2f}%''')

TOTAL ROWS: 224526


index: 0 null values, 0.00%
STATEFIP: 0 null values, 0.00%
COUNTYFIP: 0 null values, 0.00%
INDNAICS: 0 null values, 0.00%
PERWT: 0 null values, 0.00%
INCWAGE: 0 null values, 0.00%
2018 Onward ACS/PRCS INDNAICS CODE: 0 null values, 0.00%
Industry Title: 0 null values, 0.00%


In [73]:
for column in ipums_merged.columns.values:
    print(f'''{column}: {len(ipums_merged[column].unique())} unique values''')

index: 224526 unique values
STATEFIP: 1 unique values
COUNTYFIP: 35 unique values
INDNAICS: 269 unique values
PERWT: 804 unique values
INCWAGE: 889 unique values
2018 Onward ACS/PRCS INDNAICS CODE: 269 unique values
Industry Title: 269 unique values


### Merging on counties

In [74]:
ipums_w_counties = pd.merge(ipums_merged, county_info, on='COUNTYFIP', how='left')

In [75]:
print(len(ipums_merged), len(county_info), ipums_w_counties.shape)

224526 58 (224526, 20)


In [76]:
ipums_w_col = pd.merge(ipums_w_counties, cost_of_living, left_on='CDI Regions', right_on = 'Regions', how='left')

In [77]:
print(ipums_w_counties.shape, cost_of_living.shape, ipums_w_col.shape)

(224526, 20) (11, 2) (224526, 22)


In [78]:
ipums_w_col.head()

,index,STATEFIP,COUNTYFIP,INDNAICS,PERWT,INCWAGE,2018 Onward ACS/PRCS INDNAICS CODE,Industry Title,FIPS,County,...,EDD County,Census County,Population - Households,Rural/Urban,Redstone Regions,WF Regions,CDI Regions,Population,Regions,Cost of Living
0,156866,6,37,4853,21.0,23100,4853,Taxi and limousine service,6037.0,Los Angeles,...,Los Angeles County,"Los Angeles County, California","2,207,265",Urban,Los Angeles,Greater Los Angeles,Los Angeles,"10,081,570",Los Angeles,35245.0
1,157639,6,37,4853,11.0,28000,4853,Taxi and limousine service,6037.0,Los Angeles,...,Los Angeles County,"Los Angeles County, California","2,207,265",Urban,Los Angeles,Greater Los Angeles,Los Angeles,"10,081,570",Los Angeles,35245.0
2,157919,6,75,4853,4.0,1000,4853,Taxi and limousine service,6075.0,San Francisco,...,San Francisco County,"San Francisco County, California","172,190",Urban,Bay Area,Bay Area,Bay Area,"874,961",Bay Area,40083.0
3,158355,6,75,4853,147.0,1000,4853,Taxi and limousine service,6075.0,San Francisco,...,San Francisco County,"San Francisco County, California","172,190",Urban,Bay Area,Bay Area,Bay Area,"874,961",Bay Area,40083.0
4,158357,6,37,4853,35.0,28000,4853,Taxi and limousine service,6037.0,Los Angeles,...,Los Angeles County,"Los Angeles County, California","2,207,265",Urban,Los Angeles,Greater Los Angeles,Los Angeles,"10,081,570",Los Angeles,35245.0


In [79]:
ipums_regions = ipums_w_col[['INDNAICS',
                                          'INCWAGE',
                                          'Industry Title',
                                          'County',
                                          'CDI Regions',
                                          'PERWT', 'Cost of Living']]

In [80]:
ipums_regions = ipums_regions.dropna(subset=['Cost of Living'])

### Adjust for cost of living

In [81]:
ca_cost_of_living = 32466

In [32]:
# create cost of living weight
# ipums_regions['col_factor'] = ipums_regions['Cost of Living'] / ca_cost_of_living

### Statewide Industry % - wage > state mean

In [33]:
# now using normalized wage
# ipums_regions['ca_wage_thresh'] = ca_cost_of_living * ipums_regions['col_factor']

Person weight, not normalized mean wage for comparison:

In [34]:
# ca_wt_mean_wage = \
#     (sum([row['INCWAGE'] * row['PERWT'] for index, row in ipums_regions.iterrows()])) / \
#     ipums_regions['PERWT'].sum()
# ca_wt_mean_wage

50408.48252383878

Unweighted median and mean wage for comparison:

In [35]:
ca_median_wage = ipums_regions['INCWAGE'].median()
ca_median_wage

30000.0

In [36]:
ca_mean_wage = ipums_regions['INCWAGE'].mean()
ca_mean_wage

53080.0238519243

**11/29 Update: Using cost of living as threshold**

In [82]:
# same code because INCWAGE accounts for PERWT
# ipums_regions['Above CA Threshold'] = ipums_regions['INCWAGE'] > ipums_regions['ca_wage_thresh']
ipums_regions['Above CA Threshold'] = ipums_regions['INCWAGE'] > ca_cost_of_living

In [83]:
ipums_regions['wt_ind_counts'] = ipums_regions['PERWT'].groupby(ipums_regions['Industry Title']).transform('sum')

In [84]:
# cast boolean as 1/0
ipums_regions["Above CA Threshold"] = ipums_regions["Above CA Threshold"].astype(int)

In [85]:
ipums_regions["wt_CA_above_thresh"] = ipums_regions["Above CA Threshold"] * ipums_regions['PERWT']

In [86]:
ipums_regions['wt_CA_high_wage_count'] = ipums_regions['wt_CA_above_thresh'].groupby(
    ipums_regions['Industry Title']).transform('sum')

In [87]:
ipums_regions['wt_CA_high_wage_perc'] = (ipums_regions['wt_CA_high_wage_count'] / ipums_regions['wt_ind_counts']) * 100

In [88]:
ipums_regions['unweighted_ind_counts'] = ipums_regions['Industry Title'].groupby(ipums_regions['Industry Title']).transform('count')

In [90]:
high_wage_ca_wt = ipums_regions.drop_duplicates(subset='Industry Title')
high_wage_ca_wt = high_wage_ca_wt[['Industry Title', 'unweighted_ind_counts','wt_ind_counts', 'wt_CA_high_wage_count', 'wt_CA_high_wage_perc']]
high_wage_ca_wt = high_wage_ca_wt.sort_values(by='wt_CA_high_wage_perc', ascending=False)

In [91]:
high_wage_ca_wt = high_wage_ca_wt.reset_index().iloc[:,1:]

In [48]:
# high_wage_ca_wt.to_feather('data/high_wage_ca_wt.fea')

In [49]:
# high_wage_ca_wt.to_csv('data/high_wage_ca_wt.csv', index=False)

In [92]:
pd.set_option('display.max_rows', None)
high_wage_ca_wt

,Industry Title,unweighted_ind_counts,wt_ind_counts,wt_CA_high_wage_count,wt_CA_high_wage_perc
0,Coal mining,3,713.0,713.0,100.000000
1,Tobacco,1,55.0,55.0,100.000000
2,Pipeline transportation,16,1672.0,1672.0,100.000000
3,"Engine, turbine, and power transmission equipm...",52,6383.0,5722.0,89.644368
4,Software publishers,403,40200.0,35362.0,87.965174
5,Sewage treatment facilities,100,9362.0,8090.0,86.413160
6,"Electric and gas, and other combinations",340,29657.0,25471.0,85.885288
7,Internet publishing and broadcasting and web s...,890,83933.0,70992.0,84.581750
8,Not specified utilities,40,3206.0,2711.0,84.560200
9,Nonmetallic mineral mining and quarrying,58,4867.0,4038.0,82.966920


### Breakdown by Regions Pipeline

In [93]:
regions = ipums_regions['CDI Regions'].unique()
regions

array(['Los Angeles', 'Bay Area', 'Central Coast', 'Central Valley',
       'Inland Empire', 'Orange', 'Sacramento', 'San Diego-Imperial',
       'Shasta / Cascades', 'Redwood Coast'], dtype=object)

In [94]:
df_LA = ipums_regions.loc[ipums_regions['CDI Regions'] == regions[0]].copy()
df_Bay_Area = ipums_regions.loc[ipums_regions['CDI Regions'] == regions[1]].copy()
df_Cent_Coast = ipums_regions.loc[ipums_regions['CDI Regions'] == regions[2]].copy()
df_Cent_Valley = ipums_regions.loc[ipums_regions['CDI Regions'] == regions[3]].copy()
df_IE = ipums_regions.loc[ipums_regions['CDI Regions'] == regions[4]].copy()
df_OC = ipums_regions.loc[ipums_regions['CDI Regions'] == regions[5]].copy()
df_Sac = ipums_regions.loc[ipums_regions['CDI Regions'] == regions[6]].copy()
df_SD = ipums_regions.loc[ipums_regions['CDI Regions'] == regions[7]].copy()
df_Shasta = ipums_regions.loc[ipums_regions['CDI Regions'] == regions[8]].copy()
# df_nan = ipums_regions.loc[ipums_regions['CDI Regions'] == regions[9]].copy()
df_Redwood_Coast = ipums_regions.loc[ipums_regions['CDI Regions'] == regions[9]].copy()

In [95]:
region_dfs = [df_LA, df_Bay_Area, df_Cent_Coast, df_Cent_Valley, df_IE, df_OC,
              df_Sac, df_SD, df_Shasta, df_Redwood_Coast]

In [96]:
def add_to_region_df(df):
#     df['region_wage_thresh'] = \
#         (sum([row['norm_wage'] * row['PERWT'] for index, row in df.iterrows()])) / df['PERWT'].sum()
    df['above_region_thresh'] = df['INCWAGE'] > df['Cost of Living']
    df['wt_reg_ind_counts'] = df['PERWT'].groupby(df['Industry Title']).transform('sum')
    df["above_region_thresh"] = df["above_region_thresh"].astype(int)
    df["wt_reg_above_thresh"] = df["above_region_thresh"] * df['PERWT']
    df['wt_reg_high_wage_count'] = df['wt_reg_above_thresh'].groupby(
        df['Industry Title']).transform('sum')
    df['wt_reg_high_wage_perc'] = (df['wt_reg_high_wage_count'] / df['wt_reg_ind_counts']) * 100
    return df

In [97]:
for df in region_dfs:
    df = add_to_region_df(df)

### Region Industry % - wage > region mean

In [98]:
def get_region_high_wage(df, new_df):
    new_df = df.drop_duplicates(subset='Industry Title')
    new_df = new_df[['CDI Regions', 'Industry Title', 'Cost of Living', 'unweighted_ind_counts',
                     'wt_reg_ind_counts', 'wt_reg_high_wage_count', 'wt_reg_high_wage_perc']]
    new_df = new_df.sort_values(by='wt_reg_high_wage_perc', ascending=False)
    new_df = new_df.reset_index().iloc[:,1:]
    return new_df

In [99]:
# initialize new dfs
df_LA_high_wage = df_LA.copy()
df_Bay_Area_high_wage = df_Bay_Area.copy()
df_Cent_Coast_high_wage = df_Cent_Coast.copy()
df_Cent_Valley_high_wage = df_Cent_Valley.copy()
df_IE_high_wage = df_IE.copy()
df_OC_high_wage = df_OC.copy()
df_Sac_high_wage = df_Sac.copy()
df_SD_high_wage = df_SD.copy()
df_Shasta_high_wage = df_Shasta.copy()
# df_nan_high_wage = df_nan.copy()
df_Redwood_Coast_high_wage = df_Redwood_Coast.copy()

In [100]:
region_high_wage_dfs = [df_LA_high_wage, df_Bay_Area_high_wage, df_Cent_Coast_high_wage, 
                        df_Cent_Valley_high_wage, df_IE_high_wage, df_OC_high_wage,
              df_Sac_high_wage, df_SD_high_wage, df_Shasta_high_wage, df_Redwood_Coast_high_wage]

In [101]:
for i in range(len(region_dfs)):
    region_high_wage_dfs[i] = get_region_high_wage(region_dfs[i], region_dfs[i].copy())

In [102]:
regions_high_wage_merged = pd.concat(region_high_wage_dfs)

In [62]:
# regions_high_wage_merged.reset_index().iloc[:,1:].to_feather('data/regions_high_wage_merged.fea')

In [63]:
# regions_high_wage_merged.reset_index().iloc[:,1:].to_csv('data/regions_high_wage_merged.csv', index=False)

In [103]:
regions_high_wage_merged = regions_high_wage_merged.groupby(['CDI Regions','Industry Title']).agg(lambda x: x)

In [104]:
regions_high_wage_merged = regions_high_wage_merged.sort_values(['CDI Regions', 'wt_reg_high_wage_perc'], ascending=(True, False))

In [105]:
pd.set_option('display.max_rows', None)
regions_high_wage_merged

Cost of Living  \
CDI Regions        Industry Title                                                       
Bay Area           Engine, turbine, and power transmission equipme...         40083.0   
                   Knitting fabric mills, and apparel knitting mills          40083.0   
                   Metal ore mining                                           40083.0   
                   Miscellaneous paper and pulp products                      40083.0   
                   Miscellaneous petroleum and coal products                  40083.0   
                   Nonferrous metal, except aluminum, production a...         40083.0   
                   Not specified metal industries                             40083.0   
                   Railroad rolling stock manufacturing                       40083.0   
                   Tires                                                      40083.0   
                   Veneer, plywood, and engineered wood products              40083.0   
                   Nonmetallic mineral mining and quarrying                   40083.0   
                   Sawmills and wood preservation                             40083.0   
                   Cement, concrete, lime, and gypsum products man...         40083.0   
                   Internet publishing and broadcasting and web se...         40083.0   
                   Farm supplies merchant wholesalers                         40083.0   
                   Software publishers                                        40083.0   
                   Forestry, except logging                                   40083.0   
                   Pharmaceuticals and medicines                              40083.0   
                   Commercial, industrial, and other intangible as...         40083.0   
                   Data processing, hosting, and related services             40083.0   
                   Footwear Manufacturing                                     40083.0   
                   Other information services, except libraries an...         40083.0   
                   Computer systems design and related services               40083.0   
                   Public finance activities                                  40083.0   
                   Electric and gas, and other combinations                   40083.0   
                   Computer and peripheral equipment manufacturing            40083.0   
                   Construction, and mining and oil and gas field ...         40083.0   
                   Metal forgings and stampings                               40083.0   
                   Management of companies and enterprises                    40083.0   
                   Not specified utilities                                    40083.0   
                   Communications, audio, and video equipment                 40083.0   
                   Aerospace products and parts manufacturing                 40083.0   
                   Labor unions                                               40083.0   
                   Sewage treatment facilities                                40083.0   
                   Scientific research and development services               40083.0   
                   Paint, coating, and adhesives                              40083.0   
                   Petroleum refining                                         40083.0   
                   Glass and glass products manufacturing                     40083.0   
                   Wired telecommunications carriers                          40083.0   
                   Administration of economic programs and space r...         40083.0   
                   Commercial and service industry machinery manuf...         40083.0   
                   Resin, synthetic rubber, and fibers and filaments          40083.0   
                   Aircraft and parts manufacturing                           40083.0   
                   Navigational, measuring, electromedical, and co